In [1]:
import torch 
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.utils import logging

In [2]:
# Disable warnings about padding_side that cannot be rectified with current software:
logging.set_verbosity_error()

model_names = ["microsoft/DialoGPT-small", "microsoft/DialoGPT-medium", "microsoft/DialoGPT-large"]
use_model_index = 2  # Change 0: small model, 1: medium, 2: large model (requires most resources!)
model_name = model_names[use_model_index]
          
tokenizer = AutoTokenizer.from_pretrained(model_name) # , padding_side='left')
model = AutoModelForCausalLM.from_pretrained(model_name)

In [3]:
# The chat function: received a user input and chat-history and returns the model's reply and chat-history:
def reply(input_text, history=None):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input_text + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([history, new_user_input_ids], dim=-1) if history is not None else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    return tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True), chat_history_ids

In [ ]:
history = None
while True:
    input_text = input("> ")
    if input_text in ["", "bye", "quit", "exit"]:
        break
    reply_text, history_new = reply(input_text, history)
    history=history_new
    if history.shape[1]>80:
        old_shape = history.shape
        history = history[:,-80:]
        print(f"History cut from {old_shape} to {history.shape}")
    # history_text = tokenizer.decode(history[0])
    # print(f"Current history: {history_text}")
    print(f"D_GPT: {reply_text}")

>  Hi!


Current history: Hi!<|endoftext|>Hiya!<|endoftext|>
D_GPT: Hiya!


>  What's up?


Current history: Hi!<|endoftext|>Hiya!<|endoftext|>What's up?<|endoftext|>Nothing much, you?<|endoftext|>
D_GPT: Nothing much, you?


>  Here neither, pretty boring!


...

History cut from torch.Size([1, 91]) to torch.Size([1, 80])
Current history:  don't like it either.<|endoftext|>So, shall we play a game?<|endoftext|>I don't know.<|endoftext|>Do you know chess?<|endoftext|>I don't know anything about it.<|endoftext|>Do you know math?<|endoftext|>I don't know anything about it.<|endoftext|>How much is 2 plus 2?<|endoftext|>I don't know anything about it.<|endoftext|>How's the weather?<|endoftext|>It's cloudy.<|endoftext|>
D_GPT: It's cloudy.


>  Here it's sunny.


History cut from torch.Size([1, 95]) to torch.Size([1, 80])
Current history: <|endoftext|>I don't know.<|endoftext|>Do you know chess?<|endoftext|>I don't know anything about it.<|endoftext|>Do you know math?<|endoftext|>I don't know anything about it.<|endoftext|>How much is 2 plus 2?<|endoftext|>I don't know anything about it.<|endoftext|>How's the weather?<|endoftext|>It's cloudy.<|endoftext|>Here it's sunny.<|endoftext|>I don't know anything about it.<|endoftext|>
D_GPT: I don't know anything about it.


>  What temperature do you have?


History cut from torch.Size([1, 96]) to torch.Size([1, 80])
Current history:  know anything about it.<|endoftext|>Do you know math?<|endoftext|>I don't know anything about it.<|endoftext|>How much is 2 plus 2?<|endoftext|>I don't know anything about it.<|endoftext|>How's the weather?<|endoftext|>It's cloudy.<|endoftext|>Here it's sunny.<|endoftext|>I don't know anything about it.<|endoftext|>What temperature do you have?<|endoftext|>I don't know anything about it.<|endoftext|>
D_GPT: I don't know anything about it.
